In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
from sklearn.metrics import classification_report

2024-03-31 09:18:26.886427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 09:18:26.886447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 09:18:26.886969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed = 42
dataset_path = '/home/deeplearning/AI/datasets/OpenWaste/Experimental-big-9 (copy)/'
epochs = 50
batch_size = 16
lr = 0.01
weight_decay = None

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=seed),
    tf.keras.layers.RandomRotation(factor=0.05, seed=seed),
    tf.keras.layers.RandomContrast(factor=0.2, seed=seed)
])

In [5]:
train_ds_raw = tf.keras.utils.image_dataset_from_directory(dataset_path + '/train', batch_size=batch_size)
train_ds = train_ds_raw.map(lambda img, label: (data_augmentation(img), label)).shuffle(64, seed=seed).prefetch(tf.data.AUTOTUNE)

Found 132 files belonging to 9 classes.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size)

Found 153 files belonging to 9 classes.


In [7]:
class_names = train_ds_raw.class_names
num_classes = len(class_names)

In [8]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B2(
    weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.momentum = 0.90

In [9]:
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, weight_decay=weight_decay)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/tmp.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)
    
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[model_checkpoint_callback])

Epoch 1/50


2024-03-31 09:18:34.884910: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1711891116.975318  637326 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


9/9 [==============================] - 14s 319ms/step - loss: 2.1355 - accuracy: 0.1742 - val_loss: 1.8892 - val_accuracy: 0.4314
Epoch 2/50
9/9 [==============================] - 2s 148ms/step - loss: 1.5738 - accuracy: 0.6439 - val_loss: 1.4373 - val_accuracy: 0.6732
Epoch 3/50
9/9 [==============================] - 2s 143ms/step - loss: 0.9716 - accuracy: 0.8485 - val_loss: 1.0985 - val_accuracy: 0.6928
Epoch 4/50
9/9 [==============================] - 2s 149ms/step - loss: 0.5469 - accuracy: 0.9318 - val_loss: 0.9152 - val_accuracy: 0.7451
Epoch 5/50
9/9 [==============================] - 1s 92ms/step - loss: 0.2840 - accuracy: 0.9697 - val_loss: 0.8363 - val_accuracy: 0.7255
Epoch 6/50
9/9 [==============================] - 1s 92ms/step - loss: 0.1679 - accuracy: 0.9924 - val_loss: 0.8265 - val_accuracy: 0.7451
Epoch 7/50
9/9 [==============================] - 1s 88ms/step - loss: 0.1520 - accuracy: 0.9697 - val_loss: 0.8391 - val_accuracy: 0.7386
Epoch 8/50
9/9 [=================

In [10]:
saved_model_path = 'models/tmp.keras' #model_checkpoint_callback.filepath
saved_model = tf.keras.models.load_model(saved_model_path)
saved_model.evaluate(val_ds)

10/10 [==============================] - 2s 14ms/step - loss: 0.6931 - accuracy: 0.7908


[0.6931175589561462, 0.7908496856689453]